In [425]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [426]:
datafile = "C:/Users/user1/Downloads/matala2_cosmetics_2019-Nov/matala2_cosmetics_2019-Nov.csv"

In [427]:
df = pdread_csv(datafile)
df.shap.e

C:\Users\user1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# q1

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])
df = df.sort_values(['user_session', 'event_time'])
df['duration_to_next_event'] = (df.groupby('user_session')['event_time'].diff(-1) * -1).dt.seconds
df.loc[df.groupby('user_session')['event_time'].idxmax(), 'duration_to_next_event'] = 0
df.head()

# q2

In [ ]:
df = df.sort_values(by=['user_id', 'event_time'])
df['time_diff'] = df.groupby('user_id')['event_time'].diff().dt.days.apply(lambda x: 1 if x > 5 else 0)
df['funnel_number'] = df.groupby('user_id')['time_diff'].cumsum()
df['funnel_number'] = df['funnel_number'] +1
df.drop('time_diff', axis = 1, inplace = True)
df

# q3

In [ ]:
df = df.sort_values(['funnel_number', 'user_id'])
df['index_in_funnel'] = 1
df['index_in_funnel'] = df.groupby(['user_id','funnel_number'] )['index_in_funnel'].cumsum()
df['funnel_number'].value_counts()
df

# q4

In [ ]:
price_pattern = re.compile(r'\d+\.\d+')
df['price'] = df['price'].astype(str).apply(lambda x: float(price_pattern.findall(x)[0]) if price_pattern.findall(x) else None)
print(df['price'].dtype)
df

# q5

In [ ]:
type_counts = df['event_type'].value_counts()
plt.title('Number of Businesses by Type')
plt.xlabel('Type')
plt.ylabel('Count')
colors = ['red', 'green', 'blue', 'orange']
type_counts.plot.bar(color=colors)
plt.show()

# q6

In [ ]:
session_data = df[['user_id', 'user_session', 'funnel_number', 'index_in_funnel' ]]#.groupby('user_session')
session_data

In [ ]:
#df['event_type'] = df[df['event_type'].isin(['view', 'cart', 'purchase'])]
data = df.groupby(['user_session', 'event_type']).agg(
{
    'index_in_funnel' : "last",
    'product_id': lambda x: list(x),
    
}).reset_index()


pivot = data.pivot_table(index='user_session', columns= 'event_type', values='product_id',aggfunc='first')
session_data = pd.merge(session_data, pivot, on='user_session')
session_data.rename(columns={'view': 'list_of_viewed', 'cart': 'list_of_added_to_cart ', 'remove_from_cart': 'remove_from_cart'}, inplace=True)
session_data['visit_duration'] = df.groupby(['funnel_number', 'user_session'])[["duration_to_next_event"]].transform('sum')
session_data['total_event'] = df.groupby('user_session')[['event_type']].transform('count')
session_data